In [155]:
import numpy as np
import scipy.sparse as sp
import torch

In [156]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [157]:
def feature_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [158]:
def adj_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1)) # Sum each row
    r_inv = np.power(rowsum, -1/2).flatten() # Negative square root
#     r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv) # Create diagonal matrix

    # D^(-1/2).A.D^(-1/2)
    mx = r_mat_inv.dot(mx)
    mx = mx.dot(r_mat_inv)
    return mx

In [159]:
def limit_neighbors(adj, max_neighbors):
    modified_adj = np.copy(adj)
    for node in range(adj.shape[0]):
        neighbors = np.nonzero(adj[node])[0]
        if len(neighbors) > max_neighbors:
            np.random.shuffle(neighbors)
            neighbors_to_remove = neighbors[max_neighbors:]
            modified_adj[node, neighbors_to_remove] = 0
            modified_adj[neighbors_to_remove, node] = 0
    return modified_adj

In [160]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [161]:
def load_data(path="/content/data/cora/", dataset="cora"):

    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32) # Processing features into a sparse matrix
    labels = encode_onehot(idx_features_labels[:, -1]) # one-hot encoding the labels


    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32) # Reading node-ids
    idx_map = {j: i for i, j in enumerate(idx)} # Creating index for nodes to map it in adjacency matrix

    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32) # Reading edges
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape) # Mapping node-ids in the edge list to the index

    # Build adjacency matrix
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # CHECK OUT THE DIFFERENCES BETWEEN csr_matrix (features) and coo_matrix (adj)

    # Normalizing features
    features = feature_normalize(features)

#     # build symmetric adjacency matrix
#     adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

#     Normalizing the adjacency matrix after adding self loops
    adj = adj_normalize(adj + sp.eye(adj.shape[0]))

    # Setting training, validation, and test range
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    # Converting all matrices into pytorch tensors
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [162]:
# Function to find accuracy from two tensors
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels) # Get the index of maximum value of 1 dimension and typecast to labels datatype
    correct = preds.eq(labels).double() # Convert into double
    correct = correct.sum() # Sum correct predictions
    return correct / len(labels)

In [163]:
import math

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

# Class to define a neural network layer that inherits PyTorch Module
# Check out documentaion of the base class 'Module' at:
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class GraphConvolution(Module):
    # Each layer requires no. of input features, no. of output features, and optional bias
    def __init__(self, in_feat, out_feat, bias=True):
        super(GraphConvolution, self).__init__()

        self.in_features = in_feat
        self.out_features = out_feat

        # Using Parameter to automatically add weights and bias to learnable parameters
        #THIS WILL BE USEFUL ONLY WHEN WE USE Module in the model
        self.weight = Parameter(torch.FloatTensor(in_feat, out_feat))

        if bias:
            self.bias = Parameter(torch.FloatTensor(out_feat))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    # Function to get uniform distribution of weights and bias values
    # Can be removed if necessary
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    # Forward function where it actually requires the input data and operations
    def forward(self, inp, adj):
        # Basically we multiply A.H,W
        support = torch.mm(inp, self.weight)
        output = torch.spmm(adj, support)

        # Adding bias if true
        if self.bias is not None:
            return output + self.bias
        else:
            return output

In [164]:
import torch.nn as nn
import torch.nn.functional as F

# Class to define the model architecture
class GCN(nn.Module):
    # The model needs no. of input features, no. of hidden units,
    # no. of classes, and optional dropout

    # NOTE: We use a simply model with one hidden layer
        # Architecture will change for deep models
        # Ideally, we keep only a few layers in most GNNs
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        # Defining one hidden layer and one output layer
        self.gcn1 = GraphConvolution(nfeat, nhid)
        self.gcn2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    # Similar to GraphConvolution, we give required input data to the forward function
    # And specify operations - here it is activation and dropout
    def forward(self, x, adj):
        x = F.relu(self.gcn1(x, adj)) # Applying non-linearity on hidden layer 1
        # Checkout difference between nn.Dropout() and F.dropout()
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gcn2(x, adj)
        return F.log_softmax(x, dim=1) # Applying lograthmic softmax on output layer

In [165]:
import torch.optim as optim

# Loading data
adj, features, labels, idx_train, idx_val, idx_test = load_data()




# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)


# Using Adam optimizer. Other optimizer can be used too
optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

Loading cora dataset...


In [166]:
# Code for GPU computing

# CHANGE THIS CODE TO SUIT YOUR VERSION OF PYTORCH. THE SYNTAX OF THIS COULD VARY SIGNIFICANTLY

# If cuda is available, movie all data to gpu

# Set device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model and tensors to the chosen device
model = model.to(device)
features = features.to(device)
adj = adj.to(device) # Make sure you move the adjacency matrix to the correct device
labels = labels.to(device)
idx_train = idx_train.to(device)
idx_val = idx_val.to(device)
idx_test = idx_test.to(device)

# And preparing for CUDA operations
if torch.cuda.is_available():
    model.cuda()
    features = features.cuda()
    adj_layer_1 = adj_layer_1.cuda()
    adj_layer_2 = adj_layer_2.cuda()
    labels = labels.cuda()
    idx_train.cuda()
    idx_val.cuda()
    idx_test.cuda()

In [167]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score

def train(epoch, model, optimizer, features, adj, labels, train_ids):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[train_ids], labels[train_ids])
    acc_train = accuracy(output[train_ids], labels[train_ids])
    loss_train.backward()
    optimizer.step()

    model.eval()
    output = model(features, adj)
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])

    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [168]:
def test(model, features, adj, labels, idx_test):
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])

    _, predicted = torch.max(output[idx_test], 1)
    precision = precision_score(labels[idx_test].cpu(), predicted.cpu(), average='macro')
    recall = recall_score(labels[idx_test].cpu(), predicted.cpu(), average='macro')
    f1 = f1_score(labels[idx_test].cpu(), predicted.cpu(), average='macro')

    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()),
          "precision= {:.4f}".format(precision),
          "recall= {:.4f}".format(recall),
          "F1= {:.4f}".format(f1))

In [169]:
# Train and test the model
for epoch in range(200):
    train(epoch, model, optimizer, features, adj, labels, idx_train)

print()
print('Optimization finished...')



test(model, features, adj, labels, idx_test)

Epoch: 0001 loss_train: 1.9718 acc_train: 0.0786 loss_val: 1.9485 acc_val: 0.0833 time: 0.0086s
Epoch: 0002 loss_train: 1.9526 acc_train: 0.0786 loss_val: 1.9333 acc_val: 0.0833 time: 0.0053s
Epoch: 0003 loss_train: 1.9343 acc_train: 0.0857 loss_val: 1.9196 acc_val: 0.0867 time: 0.0051s
Epoch: 0004 loss_train: 1.9222 acc_train: 0.1143 loss_val: 1.9066 acc_val: 0.0900 time: 0.0052s
Epoch: 0005 loss_train: 1.8991 acc_train: 0.1714 loss_val: 1.8943 acc_val: 0.1767 time: 0.0049s
Epoch: 0006 loss_train: 1.8890 acc_train: 0.2500 loss_val: 1.8828 acc_val: 0.2967 time: 0.0057s
Epoch: 0007 loss_train: 1.8605 acc_train: 0.3286 loss_val: 1.8720 acc_val: 0.3567 time: 0.0049s
Epoch: 0008 loss_train: 1.8669 acc_train: 0.2857 loss_val: 1.8617 acc_val: 0.3700 time: 0.0050s
Epoch: 0009 loss_train: 1.8323 acc_train: 0.3500 loss_val: 1.8521 acc_val: 0.3800 time: 0.0051s
Epoch: 0010 loss_train: 1.8326 acc_train: 0.3571 loss_val: 1.8432 acc_val: 0.3800 time: 0.0050s
Epoch: 0011 loss_train: 1.8083 acc_train

###